# Module Installation

In [ ]:
!pip install redis pandas kaggle python-dotenv

# Data Prep
Download a Kaggle menu data set, clean+format the data, and then write the data to a JSON file.

In [ ]:
import pandas as pd
import os
from dotenv import load_dotenv
import json
import redis
from redis.commands.search.field import NumericField, TagField, TextField, GeoField
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query
from redis.commands.search.aggregation import AggregateRequest, Desc
from redis.commands.search import reducers
from time import perf_counter_ns

DATASET="ahmedshahriarsakib/uber-eats-usa-restaurants-menus"
RESTAURANTS="./data/restaurants.json"

load_dotenv(override=True)
if not os.path.exists(RESTAURANTS):
    if not os.path.exists("./data/restaurants.csv") or not os.path.exists("./data/restaurant-menus.csv"):
        from kaggle.api.kaggle_api_extended import KaggleApi
        api = KaggleApi()
        api.authenticate()
        api.dataset_download_files(DATASET, path='./data', quiet=True, unzip=True)

    df = pd.read_csv("./data/restaurants.csv", usecols=['id', 'name', 'full_address', 'lat', 'lng'])
    df.dropna(inplace=True)
    df['coords'] = df['lng'].astype(str) + ' ' + df['lat'].astype(str)
    df.drop(['lat', 'lng'], axis=1, inplace=True)
    
    df_menu = pd.read_csv("./data/restaurant-menus.csv")
    mask = df_menu['restaurant_id'].isin(df['id'].tolist())
    df_menu = df_menu[mask]
    df_menu.drop(['description'], axis=1, inplace=True)
    df_menu[['price', 'currency']] = df_menu['price'].str.split(' ', expand=True)
    df_menu['price'] = pd.to_numeric(df_menu['price'], errors='coerce')
    df_menu.dropna(inplace=True)
    
    restaurants = json.loads(df.to_json(orient='records'))
    for idx, restaurant in enumerate(restaurants):
        menu = json.loads((df_menu[df_menu['restaurant_id'] == restaurant['id']]).to_json(orient='records'))
        restaurant['menu'] = []
        for item in menu:
            del item['restaurant_id']
            restaurant['menu'].append(item)
        restaurants[idx] = restaurant

    with open(RESTAURANTS, "w") as outfile:
        json.dump(restaurants,outfile)  

    print(f'Number of restaurants: {len(df)}')
    print(f'Number of menu items: {len(df_menu)}')

# Redis Load
Write the restaurant JSON objects from file into Redis into two different data structures:  serialized JSON and native JSON. 

In [ ]:
with open(RESTAURANTS) as f:
    restaurants = json.load(f)

client = redis.Redis(
    host=os.environ.get("RE_HOST"),
    port=os.environ.get("RE_PORT"),
    username=os.environ.get("RE_USER"),
    password=os.environ.get("RE_PWD"),
    ssl=os.environ.get("RE_SSL")
)
client.flushdb()
pipe = client.pipeline(transaction=False)

# String
for str_idx, restaurant in enumerate(restaurants):
    key = f'restaurant_str:{restaurant["id"]}'
    stringified_val = json.dumps(restaurant)
    if str_idx>0 and str_idx%1000 == 0:
        pipe.execute()
    pipe.set(key, stringified_val)
pipe.execute()

# JSON + Search Index
idx_def = IndexDefinition(index_type=IndexType.JSON, prefix=['restaurant_json:'])
schema = [
    NumericField('$.id', as_name='id'),
    TextField('$.name', as_name='name'),
    TextField('$.full_address', as_name='address'),
    GeoField('$.coords', as_name='coords'),
    TagField('$.menu[*].category', as_name='menu_item_category'),
    TextField('$.menu[*].name', as_name='menu_item_name'),
    NumericField('$.menu[*].price', as_name='menu_item_price')
]
client.ft('restaurant_idx').create_index(schema, definition=idx_def)

for json_idx, restaurant in enumerate(restaurants):
    key = f'restaurant_json:{restaurant["id"]}'
    if json_idx>0 and json_idx%1000 == 0:
        pipe.execute()
    pipe.json().set(key, '$', restaurant)
pipe.execute()

print(f'{str_idx + 1} strings written to Redis')
print(f'{json_idx + 1} JSON objects written to Redis')

# Scenario 1  
Hypothetical Business Problem: Given a known restaurant ID and a menu item name, find its price.

In [ ]:
restaurant_id = 1
menu_item = "Arizona Green Tea"

# String
price = ''
t1 = perf_counter_ns()
restaurant_str = client.get(f'restaurant_str:{restaurant_id}')
restaurant_json = json.loads(restaurant_str)
menu = restaurant_json['menu']
for item in menu:
    if item['name'] == menu_item:
        price = item['price'] 
        break
t2 = perf_counter_ns()
print('*** String ***')
print(f'Result: {price}')
print(f'Time: {round((t2-t1)/1e6,2)} ms')
print(f'Lines of code: 7')

# JSON
price = ''
t1 = perf_counter_ns()
query = Query(f'@id:[{restaurant_id}, {restaurant_id}]')\
    .return_field(f'$.menu[?(@.name=="{menu_item}")].price', as_field='price')
result = client.ft('restaurant_idx').search(query)
t2 = perf_counter_ns()
print('\n*** JSON ***')
print(f'Result: {result.docs[0].price}')
print(f'Time: {round((t2-t1)/1e6,2)} ms')
print(f'Lines of code: 2')

# Scenario 2 
Hypothetical Business Problem: Find the number of Papa Johns restaurants within a 100 mi radius of Madison WI

In [ ]:
'''
String Algorithm:
- Loop thru the Redis key space
- Deserialize each string to JSON
- Perform a string comparison on the restaurant name element, covering letter case variations 
- For each restaurant name string match, perform a Haversine calculation
- Update local counter for Haversine matches
'''

# JSON
madison = "-89.4008, 43.0722"
t1 = perf_counter_ns() 
query = Query(f'@name:"Papa Johns" @coords:[{madison} 100 mi]').paging(0, 0)
count = client.ft('restaurant_idx').search(query).total
t2 = perf_counter_ns()
print(f'Result: {count}')
print(f'Time: {round((t2-t1)/1e6,2)} ms')
print(f'Lines of code: 2')


# Scenario 3  
Hypothetical Business Problem: Find the Top 3 menu items by count in the State of Texas

In [ ]:
'''
String Algorithm:
- Loop thru the Redis key space
- Deserialize each string to JSON
- Perform a string comparison on the restaurant address element
- For restaurant address string matches, initiate a inner loop on the menu elements - cover letter case + stem variations
- Maintain counters for every possible menu item.  Increment applicable counter per menu element
- Sort counters, display Top 3 items
'''

# JSON
t1 = perf_counter_ns() 
request = AggregateRequest('@address:TX')\
    .group_by('@menu_item_name', reducers.count().alias('item_count'))\
    .sort_by(Desc('@item_count'))\
    .limit(0,3)
result = client.ft('restaurant_idx').aggregate(request)
t2 = perf_counter_ns()
print('\n*** JSON ***')
print(f'Results:')
for row in result.rows:
    print(f'{str(row[1], "UTF-8")}: {str(row[3], "UTF-8")}')
print(f'\nTime: {round((t2-t1)/1e6,2)} ms')
print(f'Lines of Code: 2')
